In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'config'

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Weather_Database/WeatherPy_database.csv'

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.


In [5]:
# 4a. Determine if there are any empty rows.


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


NameError: name 'clean_df' is not defined

In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    

    # 6e. Make request and retrieve the JSON data from the search. 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
     
     

NameError: name 'g_key' is not defined

In [9]:
# 7. Drop the rows where there is no Hotel Name.


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

NameError: name 'clean_hotel_df' is not defined

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

NameError: name 'clean_hotel_df' is not defined

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
